In [67]:
import zipfile
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torch.nn.utils.rnn import pad_sequence
import time
import re
import string
import nltk
from nltk.corpus import stopwords
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator






In [68]:
zip_file = 'comments2k.zip'   # Adding our zip file  to extract the information

with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall('/content')


In [69]:

# Download stopwords if not already downloaded
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def clean_text(text):
    # Convert text to lowercase
    text = text.lower()
    # Remove URLs
    text = re.sub(r'http\S+', '', text)
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    # Remove stopwords
    text = ' '.join(word for word in text.split() if word not in stop_words)
    # Remove extra whitespaces
    text = ' '.join(text.split())
    return text

def load_comments(dir_path):
    comments = []
    for filename in os.listdir(dir_path):
        file_path = os.path.join(dir_path, filename)
        if os.path.isfile(file_path):
            with open(file_path, 'r', encoding='utf-8') as f:
                comments.extend(f.readlines())
    return comments

pos_dir = '/content/comments1k_pos'
neg_dir = '/content/comments1k_neg'

positive_comments = load_comments(pos_dir)
negative_comments = load_comments(neg_dir)

positive_comments = [clean_text(comment) for comment in positive_comments]
negative_comments = [clean_text(comment) for comment in negative_comments]

texts = positive_comments + negative_comments
labels = ['Positive'] * len(positive_comments) + ['Negative'] * len(negative_comments)

data = pd.DataFrame({'text': texts, 'sentiment': labels})


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [70]:
data

,text,sentiment
0,first saw movie dad encouraged watch movie one...,Positive
1,cinematically film stinks lot acting ibr br do...,Positive
2,watched episode often tftc episode enjoyable q...,Positive
3,need honest watched enjoy show gross offensive...,Positive
4,found soso romancedrama nice ending generally ...,Positive
...,...,...
1995,contains spoilersbr br timon pumbaa watching l...,Negative
1996,good thing movie shot goldie hawn standing lit...,Negative
1997,generally rated movie boasts strong cast cleve...,Negative
1998,movie relies solely somewhat controversial ima...,Negative


In [71]:
# Convert to pandas DataFrame
data = pd.DataFrame({'text': texts, 'sentiment': labels})
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)


In [72]:
train_texts, temp_texts, train_labels, temp_labels = train_test_split(texts, labels, test_size=0.3, random_state=42)
val_texts, test_texts, val_labels, test_labels = train_test_split(temp_texts, temp_labels, test_size=0.5, random_state=42)

# Define and fit the tokenizer
tokenizer = get_tokenizer('basic_english')

def yield_tokens(data_iter):
    for text in data_iter:
        yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(train_texts), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

# Tokenize and convert texts to sequences
train_sequences = [torch.tensor([vocab[token] for token in tokenizer(text.lower())]) for text in train_texts]
val_sequences = [torch.tensor([vocab[token] for token in tokenizer(text.lower())]) for text in val_texts]
test_sequences = [torch.tensor([vocab[token] for token in tokenizer(text.lower())]) for text in test_texts]

# Pad the sequences
def pad_sequences(sequences, max_len):
    return pad_sequence(sequences, batch_first=True, padding_value=0)

max_len = max(max(len(seq) for seq in train_sequences), max(len(seq) for seq in val_sequences), max(len(seq) for seq in test_sequences))

train_padded = pad_sequences(train_sequences, max_len)
val_padded = pad_sequences(val_sequences, max_len)
test_padded = pad_sequences(test_sequences, max_len)

# Define the vocabulary size
vocab_size = len(vocab)+1

print("Vocabulary size:", vocab_size)
print("Train padded shape:", train_padded.shape)
print("Validation padded shape:", val_padded.shape)
print("Test padded shape:", test_padded.shape)

Vocabulary size: 22544
Train padded shape: torch.Size([1400, 631])
Validation padded shape: torch.Size([300, 558])
Test padded shape: torch.Size([300, 572])


In [75]:
# Encode the labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
val_labels_encoded = label_encoder.transform(val_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Convert labels to torch tensors
train_labels_tensor = torch.tensor(train_labels_encoded, dtype=torch.long)
val_labels_tensor = torch.tensor(val_labels_encoded, dtype=torch.long)
test_labels_tensor = torch.tensor(test_labels_encoded, dtype=torch.long)

# Create TensorDataset
train_data = TensorDataset(train_padded, train_labels_tensor)
val_data = TensorDataset(val_padded, val_labels_tensor)
test_data = TensorDataset(test_padded, test_labels_tensor)

# Create DataLoader
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
val_loader = DataLoader(val_data, batch_size=64, shuffle=False)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

# Hyperparameters
embedding_dim = 100  # Dimension of word embeddings
hidden_size = 256
num_classes = 2
learning_rate = 0.001
num_epochs = 20

In [84]:
# Simple Neural Network Model
class SimpleNNModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size, num_classes):
        super(SimpleNNModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.fc1 = nn.Linear(embedding_dim, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        embedded = self.embedding(x)
        embedded = embedded.mean(dim=1)  # Average embeddings over the sequence
        out = self.fc1(embedded)
        out = self.relu(out)
        out = self.fc2(out)
        return out

# RNN Model
class RNNModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size, num_classes):
        super(RNNModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        embedded = self.embedding(x)
        h0 = torch.zeros(1, x.size(0), self.rnn.hidden_size).to(x.device)
        out, _ = self.rnn(embedded, h0)
        out = self.fc(out[:, -1, :])
        return out

# LSTM Model
class LSTMModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size, num_classes):
        super(LSTMModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        embedded = self.embedding(x)
        h0 = torch.zeros(1, x.size(0), self.lstm.hidden_size).to(x.device)
        c0 = torch.zeros(1, x.size(0), self.lstm.hidden_size).to(x.device)
        out, _ = self.lstm(embedded, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

# GRU Model
class GRUModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size, num_classes):
        super(GRUModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.gru = nn.GRU(embedding_dim, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        embedded = self.embedding(x)
        h0 = torch.zeros(1, x.size(0), self.gru.hidden_size).to(x.device)
        out, _ = self.gru(embedded, h0)
        out = self.fc(out[:, -1, :])
        return out

class BiLSTM3Layers(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size, num_classes, dropout=0.2):
        super(BiLSTM3Layers, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_size, num_layers=3, batch_first=True, dropout=dropout, bidirectional=True)
        self.fc = nn.Linear(hidden_size * 2, num_classes)  # Multiply by 2 for bidirectional LSTM

    def forward(self, x):
        embedded = self.embedding(x)
        lstm_out, _ = self.lstm(embedded)
        # Concatenate the final hidden states from both directions
        lstm_out = torch.cat((lstm_out[:, -1, :hidden_size], lstm_out[:, 0, hidden_size:]), dim=1)
        output = self.fc(lstm_out)
        return output

def train_and_evaluate(model, train_loader, val_loader, test_loader, num_epochs, learning_rate):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    train_losses = []
    val_losses = []
    train_accuracies = []
    val_accuracies = []

    start_time = time.time()
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs.data, 1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()
        train_losses.append(train_loss / len(train_loader.dataset))
        train_accuracies.append(train_correct / train_total)

        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        val_preds = []
        val_labels = []
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * inputs.size(0)
                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()
                val_preds.extend(predicted.cpu().numpy())
                val_labels.extend(labels.cpu().numpy())
        val_losses.append(val_loss / len(val_loader.dataset))
        val_accuracies.append(val_correct / val_total)

    time_cost = time.time() - start_time
    val_report = classification_report(val_labels, val_preds, target_names=['Negative', 'Positive'])

    test_preds = []
    test_labels = []
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            test_preds.extend(predicted.cpu().numpy())
            test_labels.extend(labels.cpu().numpy())
    test_report = classification_report(test_labels, test_preds, target_names=['Negative', 'Positive'])

    return train_losses, val_losses, train_accuracies, val_accuracies, val_report, test_report, time_cost

# Train and evaluate Simple Neural Network
simple_nn = SimpleNNModel(vocab_size, embedding_dim, hidden_size, num_classes)
simple_nn_train_losses, simple_nn_val_losses, simple_nn_train_accuracies, simple_nn_val_accuracies, simple_nn_val_report, simple_nn_test_report, simple_nn_time_cost = train_and_evaluate(simple_nn, train_loader, val_loader, test_loader, num_epochs, learning_rate)

print("Simple NN Classification Report (Validation):")
print(simple_nn_val_report)
print("Simple NN Classification Report (Test):")
print(simple_nn_test_report)
print(f"Simple NN Time Cost: {simple_nn_time_cost:.2f} seconds")

# Train and evaluate RNN
rnn = RNNModel(vocab_size, embedding_dim, hidden_size, num_classes)
rnn_train_losses, rnn_val_losses, rnn_train_accuracies, rnn_val_accuracies, rnn_val_report, rnn_test_report, rnn_time_cost = train_and_evaluate(rnn, train_loader, val_loader, test_loader, num_epochs, learning_rate)

print("RNN Classification Report (Validation):")
print(rnn_val_report)
print("RNN Classification Report (Test):")
print(rnn_test_report)
print(f"RNN Time Cost: {rnn_time_cost:.2f} seconds")

# Train and evaluate LSTM
lstm = LSTMModel(vocab_size, embedding_dim, hidden_size, num_classes)
lstm_train_losses, lstm_val_losses, lstm_train_accuracies, lstm_val_accuracies, lstm_val_report, lstm_test_report, lstm_time_cost = train_and_evaluate(lstm, train_loader, val_loader, test_loader, num_epochs, learning_rate)

print("LSTM Classification Report (Validation):")
print(lstm_val_report)
print("LSTM Classification Report (Test):")
print(lstm_test_report)
print(f"LSTM Time Cost: {lstm_time_cost:.2f} seconds")

# Train and evaluate GRU
gru = GRUModel(vocab_size, embedding_dim, hidden_size, num_classes)
gru_train_losses, gru_val_losses, gru_train_accuracies, gru_val_accuracies, gru_val_report, gru_test_report, gru_time_cost = train_and_evaluate(gru, train_loader, val_loader, test_loader, num_epochs, learning_rate)

print("GRU Classification Report (Validation):")
print(gru_val_report)
print("GRU Classification Report (Test):")
print(gru_test_report)
print(f"GRU Time Cost: {gru_time_cost:.2f} seconds")

biLSTM = BiLSTM3Layers(vocab_size, embedding_dim, hidden_size, num_classes,dropout=0.2)
biLSTM_train_losses, biLSTM_val_losses, biLSTM_train_accuracies, biLSTM_val_accuracies, biLSTM_val_report, biLSTM_test_report, biLSTM_time_cost = train_and_evaluate(biLSTM, train_loader, val_loader, test_loader, num_epochs, learning_rate)

print("biLSTM Classification Report (Validation):")
print(biLSTM_val_report)
print("biLSTM Classification Report (Test):")
print(biLSTM_test_report)
print(f"biLSTM Time Cost: {biLSTM_time_cost:.2f} seconds")



Simple NN Classification Report (Validation):
              precision    recall  f1-score   support

    Negative       0.86      0.90      0.88       147
    Positive       0.90      0.86      0.88       153

    accuracy                           0.88       300
   macro avg       0.88      0.88      0.88       300
weighted avg       0.88      0.88      0.88       300

Simple NN Classification Report (Test):
              precision    recall  f1-score   support

    Negative       0.87      0.85      0.86       151
    Positive       0.85      0.87      0.86       149

    accuracy                           0.86       300
   macro avg       0.86      0.86      0.86       300
weighted avg       0.86      0.86      0.86       300

Simple NN Time Cost: 2.84 seconds


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif

RNN Classification Report (Validation):
              precision    recall  f1-score   support

    Negative       0.00      0.00      0.00       147
    Positive       0.51      1.00      0.68       153

    accuracy                           0.51       300
   macro avg       0.26      0.50      0.34       300
weighted avg       0.26      0.51      0.34       300

RNN Classification Report (Test):
              precision    recall  f1-score   support

    Negative       0.00      0.00      0.00       151
    Positive       0.50      1.00      0.66       149

    accuracy                           0.50       300
   macro avg       0.25      0.50      0.33       300
weighted avg       0.25      0.50      0.33       300

RNN Time Cost: 8.35 seconds


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


LSTM Classification Report (Validation):
              precision    recall  f1-score   support

    Negative       0.00      0.00      0.00       147
    Positive       0.51      1.00      0.68       153

    accuracy                           0.51       300
   macro avg       0.26      0.50      0.34       300
weighted avg       0.26      0.51      0.34       300

LSTM Classification Report (Test):
              precision    recall  f1-score   support

    Negative       1.00      0.01      0.01       151
    Positive       0.50      1.00      0.67       149

    accuracy                           0.50       300
   macro avg       0.75      0.50      0.34       300
weighted avg       0.75      0.50      0.34       300

LSTM Time Cost: 20.16 seconds
GRU Classification Report (Validation):
              precision    recall  f1-score   support

    Negative       0.49      0.99      0.65       147
    Positive       0.00      0.00      0.00       153

    accuracy                        